# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f0a00d612b0>
├── 'a' --> tensor([[-0.4709,  0.0108,  0.0881],
│                   [-0.4312,  0.8834, -1.6756]])
└── 'x' --> <FastTreeValue 0x7f0a00d61190>
    └── 'c' --> tensor([[ 2.4125, -0.1576, -1.3915,  0.1810],
                        [ 1.2059,  0.3818,  0.3374, -0.3606],
                        [-0.0494,  2.8931, -0.1089, -0.9182]])

In [4]:
t.a

tensor([[-0.4709,  0.0108,  0.0881],
        [-0.4312,  0.8834, -1.6756]])

In [5]:
%timeit t.a

101 ns ± 0.406 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f0a00d612b0>
├── 'a' --> tensor([[ 0.0615,  0.2274,  1.0933],
│                   [-0.5631, -0.1838,  0.4084]])
└── 'x' --> <FastTreeValue 0x7f0a00d61190>
    └── 'c' --> tensor([[ 2.4125, -0.1576, -1.3915,  0.1810],
                        [ 1.2059,  0.3818,  0.3374, -0.3606],
                        [-0.0494,  2.8931, -0.1089, -0.9182]])

In [7]:
%timeit t.a = new_value

103 ns ± 0.0868 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.4709,  0.0108,  0.0881],
               [-0.4312,  0.8834, -1.6756]]),
    x: Batch(
           c: tensor([[ 2.4125, -0.1576, -1.3915,  0.1810],
                      [ 1.2059,  0.3818,  0.3374, -0.3606],
                      [-0.0494,  2.8931, -0.1089, -0.9182]]),
       ),
)

In [10]:
b.a

tensor([[-0.4709,  0.0108,  0.0881],
        [-0.4312,  0.8834, -1.6756]])

In [11]:
%timeit b.a

79.8 ns ± 0.691 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.6270, -1.5082, -1.0514],
               [ 1.0890, -0.2771, -0.7660]]),
    x: Batch(
           c: tensor([[ 2.4125, -0.1576, -1.3915,  0.1810],
                      [ 1.2059,  0.3818,  0.3374, -0.3606],
                      [-0.0494,  2.8931, -0.1089, -0.9182]]),
       ),
)

In [13]:
%timeit b.a = new_value

650 ns ± 1.86 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.14 µs ± 3.35 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

12.5 µs ± 28.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

177 µs ± 4.06 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

180 µs ± 9.51 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f09194722e0>
├── 'a' --> tensor([[[-0.4709,  0.0108,  0.0881],
│                    [-0.4312,  0.8834, -1.6756]],
│           
│                   [[-0.4709,  0.0108,  0.0881],
│                    [-0.4312,  0.8834, -1.6756]],
│           
│                   [[-0.4709,  0.0108,  0.0881],
│                    [-0.4312,  0.8834, -1.6756]],
│           
│                   [[-0.4709,  0.0108,  0.0881],
│                    [-0.4312,  0.8834, -1.6756]],
│           
│                   [[-0.4709,  0.0108,  0.0881],
│                    [-0.4312,  0.8834, -1.6756]],
│           
│                   [[-0.4709,  0.0108,  0.0881],
│                    [-0.4312,  0.8834, -1.6756]],
│           
│                   [[-0.4709,  0.0108,  0.0881],
│                    [-0.4312,  0.8834, -1.6756]],
│           
│                   [[-0.4709,  0.0108,  0.0881],
│                    [-0.4312,  0.8834, -1.6756]]])
└── 'x' --> <FastTreeValue 0x7f0919472940>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

40.3 µs ± 178 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f0919472e80>
├── 'a' --> tensor([[-0.4709,  0.0108,  0.0881],
│                   [-0.4312,  0.8834, -1.6756],
│                   [-0.4709,  0.0108,  0.0881],
│                   [-0.4312,  0.8834, -1.6756],
│                   [-0.4709,  0.0108,  0.0881],
│                   [-0.4312,  0.8834, -1.6756],
│                   [-0.4709,  0.0108,  0.0881],
│                   [-0.4312,  0.8834, -1.6756],
│                   [-0.4709,  0.0108,  0.0881],
│                   [-0.4312,  0.8834, -1.6756],
│                   [-0.4709,  0.0108,  0.0881],
│                   [-0.4312,  0.8834, -1.6756],
│                   [-0.4709,  0.0108,  0.0881],
│                   [-0.4312,  0.8834, -1.6756],
│                   [-0.4709,  0.0108,  0.0881],
│                   [-0.4312,  0.8834, -1.6756]])
└── 'x' --> <FastTreeValue 0x7f0919472160>
    └── 'c' --> tensor([[ 2.4125, -0.1576, -1.3915,  0.1810],
                        [ 1.2059,  0.3818,  0.3374, -0.3606],
                 

In [23]:
%timeit t_cat(trees)

37.5 µs ± 84.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

76.2 µs ± 104 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.4709,  0.0108,  0.0881],
                [-0.4312,  0.8834, -1.6756]],
       
               [[-0.4709,  0.0108,  0.0881],
                [-0.4312,  0.8834, -1.6756]],
       
               [[-0.4709,  0.0108,  0.0881],
                [-0.4312,  0.8834, -1.6756]],
       
               [[-0.4709,  0.0108,  0.0881],
                [-0.4312,  0.8834, -1.6756]],
       
               [[-0.4709,  0.0108,  0.0881],
                [-0.4312,  0.8834, -1.6756]],
       
               [[-0.4709,  0.0108,  0.0881],
                [-0.4312,  0.8834, -1.6756]],
       
               [[-0.4709,  0.0108,  0.0881],
                [-0.4312,  0.8834, -1.6756]],
       
               [[-0.4709,  0.0108,  0.0881],
                [-0.4312,  0.8834, -1.6756]]]),
    x: Batch(
           c: tensor([[[ 2.4125, -0.1576, -1.3915,  0.1810],
                       [ 1.2059,  0.3818,  0.3374, -0.3606],
                       [-0.0494,  2.8931, -0.1089, -0.9182]],
         

In [26]:
%timeit Batch.stack(batches)

98.7 µs ± 354 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.4709,  0.0108,  0.0881],
               [-0.4312,  0.8834, -1.6756],
               [-0.4709,  0.0108,  0.0881],
               [-0.4312,  0.8834, -1.6756],
               [-0.4709,  0.0108,  0.0881],
               [-0.4312,  0.8834, -1.6756],
               [-0.4709,  0.0108,  0.0881],
               [-0.4312,  0.8834, -1.6756],
               [-0.4709,  0.0108,  0.0881],
               [-0.4312,  0.8834, -1.6756],
               [-0.4709,  0.0108,  0.0881],
               [-0.4312,  0.8834, -1.6756],
               [-0.4709,  0.0108,  0.0881],
               [-0.4312,  0.8834, -1.6756],
               [-0.4709,  0.0108,  0.0881],
               [-0.4312,  0.8834, -1.6756]]),
    x: Batch(
           c: tensor([[ 2.4125, -0.1576, -1.3915,  0.1810],
                      [ 1.2059,  0.3818,  0.3374, -0.3606],
                      [-0.0494,  2.8931, -0.1089, -0.9182],
                      [ 2.4125, -0.1576, -1.3915,  0.1810],
                      [ 1.2059,  

In [28]:
%timeit Batch.cat(batches)

181 µs ± 1.07 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

464 µs ± 21.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
